In [1]:
import re
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import glob
import os # 파일 정리용

class LogAnalyzer:
    def __init__(self):
        # 로그 라인 형식: "YYYY-MM-DD HH:MM:SS [LEVEL] Message"
        self.log_pattern = re.compile(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) \[(\w+)\] (.+)')
        
        # 통계 정보 저장을 위한 자료구조
        self.stats = defaultdict(int) # 레벨별 발생 횟수
        self.errors = []              # 에러 로그 상세 정보 저장 (리스트)
        self.access_times = []        # 모든 로그의 발생 시간 저장

    def parse_log_file(self, filename: str):
        """하나의 로그 파일을 파싱하여 통계 정보를 업데이트합니다."""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, 1):
                    match = self.log_pattern.match(line.strip()) # 라인 시작부터 매칭 시도
                    if match:
                        timestamp_str, level, message = match.groups()
                        # 문자열 시간 -> datetime 객체 변환
                        timestamp = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
                        
                        # 통계 업데이트
                        self.stats[level.upper()] += 1 # 레벨을 대문자로 저장
                        self.access_times.append(timestamp)
                        
                        # 에러 레벨인 경우 상세 정보 저장
                        if level.upper() in ['ERROR', 'CRITICAL']:
                            self.errors.append({
                                'line': line_num,
                                'timestamp': timestamp,
                                'level': level.upper(),
                                'message': message
                            })
        except FileNotFoundError:
            print(f"경고: 로그 파일 '{filename}'을 찾을 수 없습니다.")
        except Exception as e:
            print(f"파일 '{filename}' 파싱 중 오류 발생: {e}")

    def analyze_log_directory(self, directory: str):
        """지정된 디렉토리의 모든 .log 파일을 파싱합니다."""
        log_files = glob.glob(os.path.join(directory, '*.log'))
        if not log_files:
            print(f"디렉토리 '{directory}'에서 .log 파일을 찾을 수 없습니다.")
            return
            
        print(f"총 {len(log_files)}개의 로그 파일을 분석합니다...")
        for log_file in log_files:
            self.parse_log_file(log_file)
        print("로그 파일 분석 완료.")

    def generate_report(self):
        """수집된 정보를 바탕으로 분석 리포트를 출력합니다."""
        print("\n" + "=" * 30)
        print("   로그 분석 결과 리포트")
        print("=" * 30)
        
        # 1. 로그 레벨별 통계
        print("\n[로그 레벨별 통계]")
        if self.stats:
            # 발생 횟수가 많은 순서대로 정렬하여 출력
            sorted_stats = sorted(self.stats.items(), key=lambda item: item[1], reverse=True)
            for level, count in sorted_stats:
                print(f"  {level}: {count} 건")
        else:
            print("  분석된 로그 레벨 데이터가 없습니다.")
        
        # 2. 시간대별 로그 발생 분석
        print("\n[시간대별 로그 발생]")
        if self.access_times:
            # 각 시간대별 로그 발생 횟수 계산 (Counter 사용)
            hour_counts = Counter(t.hour for t in self.access_times)
            # 가장 많이 발생한 시간대 찾기
            if hour_counts:
                 most_common_hour = hour_counts.most_common(1)[0][0]
                 print(f"  가장 활발한 시간대: {most_common_hour}시 (로그 {hour_counts[most_common_hour]}건)")
            else:
                 print("  로그 발생 시간 데이터가 없습니다.")
        else:
            print("  분석된 로그 발생 시간이 없습니다.")

        # 3. 에러 상세 정보 (최근 5개 에러)
        print("\n[최근 에러 발생 정보]")
        if self.errors:
            print(f"  총 에러/크리티컬 로그: {len(self.errors)} 건")
            # 가장 최근 에러 5개만 출력
            recent_errors = sorted(self.errors, key=lambda x: x['timestamp'], reverse=True)[:5]
            for error in recent_errors:
                print(f"  - 라인 {error['line']} ({error['timestamp']} [{error['level']}]) : {error['message']}")
        else:
            print("  에러 로그가 발견되지 않았습니다.")
        print("=" * 30)

# --- 사용 예시 ---
# 실제 로그 파일이 필요합니다. 테스트를 위해 임시 로그 파일 생성
if __name__ == "__main__":
    # 테스트용 로그 디렉토리 및 파일 생성
    log_dir = "log_test_dir"
    os.makedirs(log_dir, exist_ok=True)
    
    log_data = [
        "2023-10-27 10:00:00 [INFO] 서버 시작",
        "2023-10-27 10:01:05 [DEBUG] DB 연결 시도",
        "2023-10-27 10:01:06 [INFO] DB 연결 성공",
        "2023-10-27 10:05:15 [WARNING] 디스크 공간 부족 경고",
        "2023-10-27 10:10:20 [INFO] 사용자 로그인: admin",
        "2023-10-27 10:11:30 [ERROR] 파일 처리 실패: file not found", # ERROR 로그
        "2023-10-27 10:15:45 [INFO] 요청 처리 완료",
        "2023-10-27 11:00:00 [INFO] 스케줄 작업 시작",
        "2023-10-27 11:05:00 [CRITICAL] 치명적인 오류 발생!",      # CRITICAL 로그
        "2023-10-27 11:10:00 [INFO] 서버 종료",
        "This is not a valid log line." # 유효하지 않은 라인
    ]
    
    log_filename1 = os.path.join(log_dir, "app_server.log")
    log_filename2 = os.path.join(log_dir, "another_service.log")
    
    with open(log_filename1, "w", encoding="utf-8") as f:
        f.write("\n".join(log_data))
    with open(log_filename2, "w", encoding="utf-8") as f:
        f.write("2023-10-27 10:02:00 [INFO] 다른 서비스 시작\n")
        f.write("2023-10-27 10:12:00 [ERROR] 데이터 처리 오류\n")
        
    # LogAnalyzer 인스턴스 생성 및 분석 실행
    analyzer = LogAnalyzer()
    analyzer.analyze_log_directory(log_dir)
    
    # 리포트 생성
    analyzer.generate_report()
    
    # 테스트 완료 후 생성된 파일 및 디렉토리 정리
    if os.path.exists(log_filename1): os.remove(log_filename1)
    if os.path.exists(log_filename2): os.remove(log_filename2)
    if os.path.exists(log_dir): os.rmdir(log_dir)
    print("\n테스트 로그 파일 및 디렉토리 정리 완료.")


총 2개의 로그 파일을 분석합니다...
로그 파일 분석 완료.

   로그 분석 결과 리포트

[로그 레벨별 통계]
  INFO: 7 건
  ERROR: 2 건
  DEBUG: 1 건
  CRITICAL: 1 건

[시간대별 로그 발생]
  가장 활발한 시간대: 10시 (로그 9건)

[최근 에러 발생 정보]
  총 에러/크리티컬 로그: 3 건
  - 라인 9 (2023-10-27 11:05:00 [CRITICAL]) : 치명적인 오류 발생!
  - 라인 2 (2023-10-27 10:12:00 [ERROR]) : 데이터 처리 오류
  - 라인 6 (2023-10-27 10:11:30 [ERROR]) : 파일 처리 실패: file not found

테스트 로그 파일 및 디렉토리 정리 완료.
